In [1]:
from os import walk
import sys, email, csv
import numpy as np 
import pandas as pd
import codecs
import sklearn as sk

In [2]:
%matplotlib inline

In [3]:
# Load Enron data
emails_df = pd.read_csv('emails.csv', parse_dates=True, engine="python", delimiter=",", encoding='utf-8', error_bad_lines=False)

Skipping line 18226: field larger than field limit (131072)
Skipping line 20082: ',' expected after '"'
Skipping line 30906: field larger than field limit (131072)
Skipping line 31519: field larger than field limit (131072)
Skipping line 32130: field larger than field limit (131072)
Skipping line 37541: field larger than field limit (131072)
Skipping line 37688: ',' expected after '"'
Skipping line 37725: ',' expected after '"'
Skipping line 37952: ',' expected after '"'
Skipping line 38004: field larger than field limit (131072)
Skipping line 38008: ',' expected after '"'
Skipping line 38011: ',' expected after '"'
Skipping line 38068: ',' expected after '"'
Skipping line 38076: ',' expected after '"'
Skipping line 38083: ',' expected after '"'
Skipping line 38085: ',' expected after '"'
Skipping line 38086: ',' expected after '"'
Skipping line 38090: ',' expected after '"'
Skipping line 38101: ',' expected after '"'
Skipping line 38108: ',' expected after '"'
Skipping line 38145: ','

Skipping line 69125: field larger than field limit (131072)
Skipping line 69159: ',' expected after '"'
Skipping line 69189: ',' expected after '"'
Skipping line 69252: ',' expected after '"'
Skipping line 69327: ',' expected after '"'
Skipping line 69354: ',' expected after '"'
Skipping line 69372: ',' expected after '"'
Skipping line 69377: ',' expected after '"'
Skipping line 69388: ',' expected after '"'
Skipping line 69395: ',' expected after '"'
Skipping line 69410: ',' expected after '"'
Skipping line 69418: ',' expected after '"'
Skipping line 69429: ',' expected after '"'
Skipping line 69439: ',' expected after '"'
Skipping line 69475: ',' expected after '"'
Skipping line 69548: ',' expected after '"'
Skipping line 69595: ',' expected after '"'
Skipping line 69597: ',' expected after '"'
Skipping line 69613: ',' expected after '"'
Skipping line 69626: ',' expected after '"'
Skipping line 69640: ',' expected after '"'
Skipping line 69642: ',' expected after '"'
Skipping line 69

Skipping line 74963: field larger than field limit (131072)
Skipping line 74980: ',' expected after '"'
Skipping line 75029: ',' expected after '"'
Skipping line 75030: ',' expected after '"'
Skipping line 75086: ',' expected after '"'
Skipping line 75095: ',' expected after '"'
Skipping line 75294: ',' expected after '"'
Skipping line 75485: ',' expected after '"'
Skipping line 75488: ',' expected after '"'
Skipping line 75491: ',' expected after '"'
Skipping line 75519: ',' expected after '"'
Skipping line 75621: ',' expected after '"'
Skipping line 75628: ',' expected after '"'
Skipping line 75663: ',' expected after '"'
Skipping line 75668: ',' expected after '"'
Skipping line 75829: field larger than field limit (131072)
Skipping line 75893: ',' expected after '"'
Skipping line 75944: ',' expected after '"'
Skipping line 75948: ',' expected after '"'
Skipping line 76036: ',' expected after '"'
Skipping line 76052: ',' expected after '"'
Skipping line 76055: ',' expected after '"'


Skipping line 83945: ',' expected after '"'
Skipping line 83959: ',' expected after '"'
Skipping line 84209: ',' expected after '"'
Skipping line 84242: ',' expected after '"'
Skipping line 84322: ',' expected after '"'
Skipping line 84427: field larger than field limit (131072)
Skipping line 84830: ',' expected after '"'
Skipping line 84834: ',' expected after '"'
Skipping line 84884: ',' expected after '"'
Skipping line 84889: ',' expected after '"'
Skipping line 84893: ',' expected after '"'
Skipping line 84897: ',' expected after '"'
Skipping line 84899: ',' expected after '"'
Skipping line 85098: ',' expected after '"'
Skipping line 85133: ',' expected after '"'
Skipping line 85134: ',' expected after '"'
Skipping line 85228: ',' expected after '"'
Skipping line 85257: ',' expected after '"'
Skipping line 85266: ',' expected after '"'
Skipping line 85274: ',' expected after '"'
Skipping line 85293: ',' expected after '"'
Skipping line 85305: ',' expected after '"'
Skipping line 85

Skipping line 92042: field larger than field limit (131072)
Skipping line 92050: ',' expected after '"'
Skipping line 92057: ',' expected after '"'
Skipping line 92059: ',' expected after '"'
Skipping line 92064: ',' expected after '"'
Skipping line 92072: ',' expected after '"'
Skipping line 92162: ',' expected after '"'
Skipping line 92189: ',' expected after '"'
Skipping line 92191: ',' expected after '"'
Skipping line 92247: field larger than field limit (131072)
Skipping line 92265: ',' expected after '"'
Skipping line 92355: ',' expected after '"'
Skipping line 92447: ',' expected after '"'
Skipping line 92518: ',' expected after '"'
Skipping line 92523: ',' expected after '"'
Skipping line 92602: ',' expected after '"'
Skipping line 92618: ',' expected after '"'
Skipping line 92657: ',' expected after '"'
Skipping line 92661: ',' expected after '"'
Skipping line 92937: ',' expected after '"'
Skipping line 93084: field larger than field limit (131072)
Skipping line 93613: ',' exp

...

Skipping line 409558: Expected 2 fields in line 409558, saw 3
Skipping line 409577: Expected 2 fields in line 409577, saw 3
Skipping line 409581: Expected 2 fields in line 409581, saw 3
Skipping line 409633: Expected 2 fields in line 409633, saw 3
Skipping line 409638: Expected 2 fields in line 409638, saw 3
Skipping line 409644: Expected 2 fields in line 409644, saw 3
Skipping line 409651: Expected 2 fields in line 409651, saw 3
Skipping line 409703: Expected 2 fields in line 409703, saw 3
Skipping line 409729: Expected 2 fields in line 409729, saw 4
Skipping line 409733: Expected 2 fields in line 409733, saw 3
Skipping line 409765: Expected 2 fields in line 409765, saw 3
Skipping line 409862: Expected 2 fields in line 409862, saw 3
Skipping line 409874: Expected 2 fields in line 409874, saw 3
Skipping line 409893: Expected 2 fields in line 409893, saw 3
Skipping line 409898: Expected 2 fields in line 409898, saw 3
Skipping line 409904: Expected 2 fields in line 409904, saw 3
Skipping

Skipping line 511763: Expected 2 fields in line 511763, saw 3
Skipping line 511782: Expected 2 fields in line 511782, saw 3
Skipping line 511783: Expected 2 fields in line 511783, saw 3
Skipping line 511819: Expected 2 fields in line 511819, saw 3
Skipping line 511838: Expected 2 fields in line 511838, saw 4
Skipping line 511839: Expected 2 fields in line 511839, saw 3
Skipping line 511840: Expected 2 fields in line 511840, saw 3
Skipping line 511841: Expected 2 fields in line 511841, saw 3
Skipping line 511842: Expected 2 fields in line 511842, saw 3
Skipping line 511843: Expected 2 fields in line 511843, saw 3
Skipping line 511844: Expected 2 fields in line 511844, saw 3
Skipping line 511845: Expected 2 fields in line 511845, saw 3
Skipping line 511846: Expected 2 fields in line 511846, saw 3
Skipping line 511847: Expected 2 fields in line 511847, saw 3
Skipping line 511848: Expected 2 fields in line 511848, saw 3
Skipping line 511849: Expected 2 fields in line 511849, saw 3
Skipping

Skipping line 559759: Expected 2 fields in line 559759, saw 8
Skipping line 559760: Expected 2 fields in line 559760, saw 4
Skipping line 559761: Expected 2 fields in line 559761, saw 4
Skipping line 559762: Expected 2 fields in line 559762, saw 4
Skipping line 559764: Expected 2 fields in line 559764, saw 4
Skipping line 559772: Expected 2 fields in line 559772, saw 3
Skipping line 559778: Expected 2 fields in line 559778, saw 3
Skipping line 559787: Expected 2 fields in line 559787, saw 4
Skipping line 559790: Expected 2 fields in line 559790, saw 4
Skipping line 559791: Expected 2 fields in line 559791, saw 3
Skipping line 559792: Expected 2 fields in line 559792, saw 4
Skipping line 559794: Expected 2 fields in line 559794, saw 3
Skipping line 559802: Expected 2 fields in line 559802, saw 3
Skipping line 559806: Expected 2 fields in line 559806, saw 5
Skipping line 559807: Expected 2 fields in line 559807, saw 3
Skipping line 559808: Expected 2 fields in line 559808, saw 3
Skipping

Skipping line 642449: Expected 2 fields in line 642449, saw 3
Skipping line 642456: Expected 2 fields in line 642456, saw 3
Skipping line 642464: Expected 2 fields in line 642464, saw 3
Skipping line 642466: Expected 2 fields in line 642466, saw 3
Skipping line 642469: Expected 2 fields in line 642469, saw 3
Skipping line 642473: Expected 2 fields in line 642473, saw 5
Skipping line 642479: Expected 2 fields in line 642479, saw 3
Skipping line 642485: Expected 2 fields in line 642485, saw 3
Skipping line 642504: Expected 2 fields in line 642504, saw 3
Skipping line 642545: Expected 2 fields in line 642545, saw 3
Skipping line 642548: Expected 2 fields in line 642548, saw 3
Skipping line 642559: Expected 2 fields in line 642559, saw 3
Skipping line 642578: Expected 2 fields in line 642578, saw 3
Skipping line 642584: Expected 2 fields in line 642584, saw 3
Skipping line 642599: Expected 2 fields in line 642599, saw 3
Skipping line 642606: Expected 2 fields in line 642606, saw 5
Skipping

Skipping line 661283: Expected 2 fields in line 661283, saw 3
Skipping line 661292: Expected 2 fields in line 661292, saw 4
Skipping line 661300: Expected 2 fields in line 661300, saw 4
Skipping line 661311: Expected 2 fields in line 661311, saw 3
Skipping line 661313: Expected 2 fields in line 661313, saw 3
Skipping line 661316: Expected 2 fields in line 661316, saw 4
Skipping line 661328: Expected 2 fields in line 661328, saw 3
Skipping line 661332: Expected 2 fields in line 661332, saw 5
Skipping line 661336: Expected 2 fields in line 661336, saw 3
Skipping line 661340: Expected 2 fields in line 661340, saw 3
Skipping line 661342: Expected 2 fields in line 661342, saw 3
Skipping line 661349: Expected 2 fields in line 661349, saw 3
Skipping line 661357: Expected 2 fields in line 661357, saw 3
Skipping line 661360: Expected 2 fields in line 661360, saw 3
Skipping line 661363: Expected 2 fields in line 661363, saw 3
Skipping line 661389: Expected 2 fields in line 661389, saw 3
Skipping

In [4]:
emails_df.shape

(843222, 2)

In [5]:
emails_df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [6]:
# Replace NA values from initial set
emails_df_NA = emails_df.replace(to_replace='None', value=np.nan).dropna()

In [7]:
emails_df_NA.shape

(554134, 2)

In [8]:
emails_df_NA.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [9]:
## Helper function 1
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

In [10]:
## Helper function 2
def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [11]:
# Map Enron email data to be added into pandas dataframe
reload(sys)
sys.setdefaultencoding('utf8')

# Parse the emails into a list email objects
messages = list(map(email.message_from_string, emails_df_NA['message']))
emails_df_NA.drop('message', axis=1, inplace=True)

# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    emails_df_NA[key] = [doc[key] for doc in messages]

# Parse content from emails
emails_df_NA['content'] = list(map(get_text_from_email, messages))

# Split multiple email addresses
emails_df_NA['From'] = emails_df_NA['From'].map(split_email_addresses)
emails_df_NA['To'] = emails_df_NA['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
emails_df_NA['user'] = emails_df_NA['file'].map(lambda x:x.split('/')[0])
del messages

In [12]:
emails_df_NA.head()

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",(phillip.allen@enron.com),(tim.belden@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p
1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",(phillip.allen@enron.com),(john.lavorato@enron.com),Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p
2,allen-p/_sent_mail/100.,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",(phillip.allen@enron.com),(leah.arsdall@enron.com),Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p
3,allen-p/_sent_mail/1000.,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",(phillip.allen@enron.com),(randall.gay@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p
4,allen-p/_sent_mail/1001.,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",(phillip.allen@enron.com),(greg.piper@enron.com),Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p


In [13]:
# Gather training tags
csvFileArray = []
outlst = []

for directory in walk("."):
    for fle in directory[2]:
        if "personal" in directory[0] and ".csv" in fle:
            csvFileArray = []
            path = directory[0] + "/" + fle
            csvfile = codecs.open(path, 'rb')
            for row in csv.reader(csvfile, delimiter = '.'):
                csvFileArray.append(row)
            dte = str(csvFileArray[0])
            dte = dte[8:-2]
            outlst.append(("personal", fle, dte))
        
        elif "business" in directory[0] and ".csv" in fle:
            csvFileArray = []
            path = directory[0] + "/" + fle
            csvfile = codecs.open(path, 'rb')
            for row in csv.reader(csvfile, delimiter = '.'):
                csvFileArray.append(row)
            dte = str(csvFileArray[0])
            dte = dte[8:-2]
            outlst.append(("business", fle, dte))
            
        elif "chain_mails" in directory[0] and ".csv" in fle:
            csvFileArray = []
            path = directory[0] + "/" + fle
            csvfile = codecs.open(path, 'rb')
            for row in csv.reader(csvfile, delimiter = "."):
                csvFileArray.append(row)
            dte = str(csvFileArray[0])
            dte = dte[8:-2]
            outlst.append(("chain_mails", fle, dte))
            
        elif "enron_online" in directory[0] and ".csv" in fle:
            csvFileArray = []
            path = directory[0] + "/" + fle
            csvfile = codecs.open(path, 'rb')
            for row in csv.reader(csvfile, delimiter = "."):
                csvFileArray.append(row)
            dte = str(csvFileArray[0])
            dte = dte[8:-2]
            outlst.append(("enron_online", fle, dte))
            
        elif "general_announcements" in directory[0] and ".csv" in fle:
            csvFileArray = []
            path = directory[0] + "/" + fle
            csvfile = codecs.open(path, 'rb')
            for row in csv.reader(csvfile, delimiter = "."):
                csvFileArray.append(row)
            dte = str(csvFileArray[0])
            dte = dte[8:-2]
            outlst.append(("general_announcements", fle, dte))
            
        elif "human_resources" in directory[0] and ".csv" in fle:
            csvFileArray = []
            path = directory[0] + "/" + fle
            csvfile = codecs.open(path, 'rb')
            for row in csv.reader(csvfile, delimiter = "."):
                csvFileArray.append(row)
            dte = str(csvFileArray[0])
            dte = dte[8:-2]
            outlst.append(("human_resources", fle, dte))
            

In [14]:
# Load training tags into pandas dataframe
tags = pd.DataFrame(list(outlst), columns=['tags','file','date'])

In [15]:
# Join the two datasets
emails_merged_full = pd.merge(emails_df_NA, tags, left_on = 'Date', right_on = 'date', how = 'inner')

In [16]:
# Number of training samples
emails_merged_full.shape

(12607, 21)

In [17]:
# Curated dataset ready for scikit-learn tuning
emails_merged_full.head()

,file_x,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,...,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user,tags,file_y,date
0,allen-p/_sent_mail/26.,<15164543.1075855378954.JavaMail.evans@thyme>,"Wed, 25 Apr 2001 16:52:00 -0700 (PDT)",(phillip.allen@enron.com),(john.lavorato@enron.com),Re: This morning's Commission meeting delayed,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,\n---------------------- Forwarded by Phillip ...,allen-p,enron_online,Ron.williams-w3.timbelden.2.csv,"Wed, 25 Apr 2001 16:52:00 -0700 (PDT)"
1,allen-p/sent_items/61.,<24466219.1075858638656.JavaMail.evans@thyme>,"Wed, 25 Apr 2001 16:52:00 -0700 (PDT)",(phillip.allen@enron.com),(john.lavorato@enron.com),Re: This morning's Commission meeting delayed,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,...,,,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",Allen-P,PALLEN (Non-Privileged).pst,\n---------------------- Forwarded by Phillip ...,allen-p,enron_online,Ron.williams-w3.timbelden.2.csv,"Wed, 25 Apr 2001 16:52:00 -0700 (PDT)"
2,williams-w3/timbelden/2.,<12767107.1075840010399.JavaMail.evans@thyme>,"Wed, 25 Apr 2001 16:52:00 -0700 (PDT)",(tim.belden@enron.com),"(robert.badeer@enron.com, bill.iii@enron.com, ...",Daily EOL/ICE Summary 4/24,1.0,text/plain; charset=us-ascii,7bit,Tim Belden <Tim Belden/HOU/ECT@ECT>,...,,,"\ExMerge - Williams III, Bill\TimBelden",WILLIAMS-W3,,The last couple of days on eol have been amazi...,williams-w3,enron_online,Ron.williams-w3.timbelden.2.csv,"Wed, 25 Apr 2001 16:52:00 -0700 (PDT)"
3,allen-p/_sent_mail/440.,<29628244.1075855725718.JavaMail.evans@thyme>,"Mon, 19 Mar 2001 00:45:00 -0800 (PST)",(phillip.allen@enron.com),(llewter@austin.rr.com),Re: Buyout,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,...,,,\Phillip_Allen_June2001\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Larrry,\n\nI realize you are disappointed abou...",allen-p,business,Ron.beck-s.apollo__beth.9.csv,"Mon, 19 Mar 2001 00:45:00 -0800 (PST)"
4,allen-p/all_documents/459.,<18425275.1075855696118.JavaMail.evans@thyme>,"Mon, 19 Mar 2001 00:45:00 -0800 (PST)",(phillip.allen@enron.com),(llewter@austin.rr.com),Re: Buyout,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,...,,,\Phillip_Allen_June2001\Notes Folders\All docu...,Allen-P,pallen.nsf,"Larrry,\n\nI realize you are disappointed abou...",allen-p,business,Ron.beck-s.apollo__beth.9.csv,"Mon, 19 Mar 2001 00:45:00 -0800 (PST)"


In [ ]:
# emails_merged_full['tags']

In [18]:
target_vals = ['business', 'general_annoucements', 'enron_online',
               'personal', 'chain_emails', 'human_resources']

In [ ]:
#emails_merged_full.tags

In [19]:
# From http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
# Count Vectorize the data

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(emails_merged_full['content'])

In [20]:
X_train_counts.shape

(12607, 38051)

In [21]:
count_vect.vocabulary_.get(u'enron')

14233

In [22]:
# Tf-idf the content
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(12607, 38051)

In [23]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(12607, 38051)

In [24]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, emails_merged_full.tags)

In [25]:
docs_new = ['Enron enron enron enron', 'Which way is the pizza shop?',
            'Setup a conference call for Monday morning.']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, tag in zip(docs_new, predicted):
    print('%r => %s' % (doc, tag))
    print(tag)

In [26]:
predicted

array(['general_announcements', 'general_announcements', 'business'],
      dtype='|S21')

In [ ]:
# building a pipeline to create a compound classifier
from sklearn.pipeline import Pipeline
#text_clf_pipeline = Pipeline([('vect', CountVectorizer()),
#                              ('fittrans', CountVectorizer().fit_transform(enron_train.content)),
#                     ('tfidf', TfidfTransformer()),
#                     ('clf', MultinomialNB()),
#])

#                              ('transform', CountVectorizer().transform(enron_train.content)),

In [27]:
# split train and test set
from sklearn.model_selection import train_test_split

enron_train, enron_test = train_test_split(emails_merged_full, test_size=0.2)

emails_test = enron_test.content

In [28]:
# Predict Values

# This "pipeline" actually works.
from sklearn.naive_bayes import MultinomialNB

# Taking the training data and turning it into a count feature vector
count_vect_2 = CountVectorizer()
X_train_counts_2 = count_vect_2.fit_transform(enron_train.content)

# Turning the vector into a weighted feature vector
tfidf_transformer_2 = TfidfTransformer()
X_train_tfidf_2 = tfidf_transformer_2.fit_transform(X_train_counts_2)

# Using a specific algo to train the model
clf_2 = MultinomialNB().fit(X_train_tfidf_2, enron_train.tags)

# Prepare the test data into a count vector
X_new_counts_2 = count_vect_2.transform(enron_test.content)
# Turning the vector into a weighted feature vector for the test data
X_new_tfidf_2 = tfidf_transformer_2.transform(X_new_counts_2)

# Making a prediction for the test data
predicted_2 = clf_2.predict(X_new_tfidf_2)

# Measure the accuracy
np.mean(predicted_2 == enron_test.tags)

0.74306106264869154

In [29]:
enron_test.shape

(2522, 21)

In [30]:
X_new_counts_2.shape

(2522, 36939)

In [31]:
enron_train.shape

(10085, 21)

In [32]:
predicted_2.shape

(2522,)

In [33]:
#LinearSupportVectorClassifier
from sklearn.svm import LinearSVC

count_vect_3 = CountVectorizer()
X_train_counts_3 = count_vect_3.fit_transform(enron_train.content)

tfidf_transformer_3 = TfidfTransformer()
X_train_tfidf_3 = tfidf_transformer_3.fit_transform(X_train_counts_3)

clf_3 = LinearSVC().fit(X_train_tfidf_3, enron_train.tags)

X_new_counts_3 = count_vect_3.transform(enron_test.content)
X_new_tfidf_3 = tfidf_transformer_3.transform(X_new_counts_3)

predicted_3 = clf_3.predict(X_new_tfidf_3)

np.mean(predicted_3 == enron_test.tags)

0.93536875495638383

In [34]:
#StocasticGradientDescentClassifier
from sklearn.linear_model import SGDClassifier

count_vect_4 = CountVectorizer()
X_train_counts_4 = count_vect_4.fit_transform(enron_train.content)

tfidf_transformer_4 = TfidfTransformer()
X_train_tfidf_4 = tfidf_transformer_4.fit_transform(X_train_counts_4)

clf_4 = SGDClassifier().fit(X_train_tfidf_4, enron_train.tags)

X_new_counts_4 = count_vect_4.transform(enron_test.content)
X_new_tfidf_4 = tfidf_transformer_4.transform(X_new_counts_4)

predicted_4 = clf_4.predict(X_new_tfidf_4)

np.mean(predicted_4 == enron_test.tags)

0.91435368754956381

In [35]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

from sklearn import metrics
report_2 = metrics.classification_report(enron_test.tags, predicted_2)
print(report_2)
# , target_names = target_vals

In [36]:
metrics.confusion_matrix(enron_test.tags, predicted_2)

array([[692,   0,   0, 102,   0,  10],
       [  2,   0,   0,   1,   0,   0],
       [ 35,   0,   6,  21,   0,   1],
       [ 61,   0,   0, 802,   0,   0],
       [147,   0,   0,  67,  35,   4],
       [149,   0,   0,  48,   0, 339]])

In [37]:
report_3 =  metrics.classification_report(enron_test.tags, predicted_3)
print(report_3)

In [38]:
metrics.confusion_matrix(enron_test.tags, predicted_3)

array([[733,   0,   2,  37,   4,  28],
       [  1,   2,   0,   0,   0,   0],
       [  6,   0,  45,  11,   0,   1],
       [ 12,   0,   0, 848,   1,   2],
       [ 20,   0,   0,   5, 223,   5],
       [ 16,   0,   0,  12,   0, 508]])

In [40]:
report_4 =  metrics.classification_report(enron_test.tags, predicted_4)
print(report_4)

In [42]:
metrics.confusion_matrix(enron_test.tags, predicted_4)

array([[746,   0,   2,  22,   3,  31],
       [  1,   2,   0,   0,   0,   0],
       [  9,   0,  42,  11,   0,   1],
       [ 39,   0,   0, 817,   1,   6],
       [ 30,   0,   0,   4, 210,   9],
       [ 36,   0,   0,  11,   0, 489]])

In [ ]:
# Searching to find the best parameters for each classification algorithm in order to improve the model
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),
}

In [ ]:
# Create a Grid Search object 
gs_clf = GridSearchCV(clf, parameters, n_jobs=-1)
gs_clf_2 = GridSearchCV(clf_2, parameters, n_jobs=-1)
gs_clf_3 = GridSearchCV(clf_3, parameters, n_jobs=-1)
gs_clf_4 = GridSearchCV(clf_4, parameters, n_jobs=-1)

In [ ]:
gs_clf_3 = gs_clf_3.fit(enron_train.content[:400], enron_train.tags[:400])

In [ ]:
clf_2.get_params().keys() #MultinomialNaiveBayes

In [ ]:
clf_3.get_params().keys() #LinearSupportVectorClassifier

In [ ]:
clf_4.get_params().keys() #StocasticGradientDescentClassifier

In [43]:
# New task for the week.
# Make sure there's no leaking within the training and test sets.

In [49]:
# Sorting the indices shows that there's no overlapping data in both the 
# training and test sets. This confirms that from sklearn.model_selection import train_test_split
# worked and validates our training runnings.
enron_train.sort_index()

,file_x,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,...,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user,tags,file_y,date
0,allen-p/_sent_mail/26.,<15164543.1075855378954.JavaMail.evans@thyme>,"Wed, 25 Apr 2001 16:52:00 -0700 (PDT)",(phillip.allen@enron.com),(john.lavorato@enron.com),Re: This morning's Commission meeting delayed,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,\n---------------------- Forwarded by Phillip ...,allen-p,enron_online,Ron.williams-w3.timbelden.2.csv,"Wed, 25 Apr 2001 16:52:00 -0700 (PDT)"
1,allen-p/sent_items/61.,<24466219.1075858638656.JavaMail.evans@thyme>,"Wed, 25 Apr 2001 16:52:00 -0700 (PDT)",(phillip.allen@enron.com),(john.lavorato@enron.com),Re: This morning's Commission meeting delayed,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,...,,,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",Allen-P,PALLEN (Non-Privileged).pst,\n---------------------- Forwarded by Phillip ...,allen-p,enron_online,Ron.williams-w3.timbelden.2.csv,"Wed, 25 Apr 2001 16:52:00 -0700 (PDT)"
2,williams-w3/timbelden/2.,<12767107.1075840010399.JavaMail.evans@thyme>,"Wed, 25 Apr 2001 16:52:00 -0700 (PDT)",(tim.belden@enron.com),"(robert.badeer@enron.com, bill.iii@enron.com, ...",Daily EOL/ICE Summary 4/24,1.0,text/plain; charset=us-ascii,7bit,Tim Belden <Tim Belden/HOU/ECT@ECT>,...,,,"\ExMerge - Williams III, Bill\TimBelden",WILLIAMS-W3,,The last couple of days on eol have been amazi...,williams-w3,enron_online,Ron.williams-w3.timbelden.2.csv,"Wed, 25 Apr 2001 16:52:00 -0700 (PDT)"
3,allen-p/_sent_mail/440.,<29628244.1075855725718.JavaMail.evans@thyme>,"Mon, 19 Mar 2001 00:45:00 -0800 (PST)",(phillip.allen@enron.com),(llewter@austin.rr.com),Re: Buyout,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,...,,,\Phillip_Allen_June2001\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Larrry,\n\nI realize you are disappointed abou...",allen-p,business,Ron.beck-s.apollo__beth.9.csv,"Mon, 19 Mar 2001 00:45:00 -0800 (PST)"
4,allen-p/all_documents/459.,<18425275.1075855696118.JavaMail.evans@thyme>,"Mon, 19 Mar 2001 00:45:00 -0800 (PST)",(phillip.allen@enron.com),(llewter@austin.rr.com),Re: Buyout,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,...,,,\Phillip_Allen_June2001\Notes Folders\All docu...,Allen-P,pallen.nsf,"Larrry,\n\nI realize you are disappointed abou...",allen-p,business,Ron.beck-s.apollo__beth.9.csv,"Mon, 19 Mar 2001 00:45:00 -0800 (PST)"
5,allen-p/sent/393.,<14758761.1075855715366.JavaMail.evans@thyme>,"Mon, 19 Mar 2001 00:45:00 -0800 (PST)",(phillip.allen@enron.com),(llewter@austin.rr.com),Re: Buyout,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,...,,,\Phillip_Allen_June2001\Notes Folders\Sent,Allen-P,pallen.nsf,"Larrry,\n\nI realize you are disappointed abou...",allen-p,business,Ron.beck-s.apollo__beth.9.csv,"Mon, 19 Mar 2001 00:45:00 -0800 (PST)"
6,beck-s/all_documents/508.,<4114555.1075849824674.JavaMail.evans@thyme>,"Mon, 19 Mar 2001 00:45:00 -0800 (PST)",(beth.apollo@enron.com),(sally.beck@enron.com),FW: CGI Proposal Update,1.0,text/plain; charset=ANSI_X3.4-1968,7bit,Beth Apollo,...,,,\Sally_Beck_Nov2001\Notes Folders\All documents,BECK-S,sbeck.nsf,I have not spoken with Steve Abbnat as of yet ...,beck-s,business,Ron.beck-s.apollo__beth.9.csv,"Mon, 19 Mar 2001 00:45:00 -0800 (PST)"
7,beck-s/apollo__beth/9.,<2970842.1075849829370.JavaMail.evans@thyme>,"Mon, 19 Mar 2001 00:45:00 -0800 (PST)",(beth.apollo@enron.com),(sally.beck@enron.com),FW: CGI Proposal Update,1.0,text/plain; charset=ANSI_X3.4-1968,7bit,Beth Apollo,...,,,"\Sally_Beck_Nov2001\Notes Folders\Apollo, beth",BECK-S,sbeck.nsf,I have not spoken with Steve Abbnat as of yet ...,beck-s,business,Ron.beck-s.apollo__beth.9.csv,"Mon, 19 Mar 2001 00:45:00 -0800 (PST)"
8,beck-s/discussion_threads/460.,<33459963.1075849845771.JavaMail.evans@thyme>,"Mon, 19 Mar 2001 00:45:00 -0800 (PST)",(beth.a

In [50]:
enron_test.sort_index()

,file_x,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,...,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user,tags,file_y,date
10,allen-p/all_documents/600.,<17326808.1075855699341.JavaMail.evans@thyme>,"Thu, 4 Jan 2001 06:41:00 -0800 (PST)",(phillip.allen@enron.com),(gallen@thermon.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,...,,,\Phillip_Allen_June2001\Notes Folders\All docu...,Allen-P,pallen.nsf,Questions about 12/29 rentroll:\n\n There were...,allen-p,business,Ron.beck-s.mpr.1.csv,"Thu, 4 Jan 2001 06:41:00 -0800 (PST)"
27,davis-d/deleted_items/102.,<19818151.1075851575966.JavaMail.evans@thyme>,"Thu, 18 Oct 2001 15:10:12 -0700 (PDT)",(no.address@enron.com),None,UPDATE - Supported Internet Email Addresses,1.0,text/plain; charset=us-ascii,7bit,Enron Global Technology@ENRON,...,,,\DDAVIS (Non-Privileged)\Deleted Items,DAVIS-D,DDAVIS (Non-Privileged).pst,"\nEarlier this week, Enron Global Technology a...",davis-d,general_announcements,Ron.farmer-d.enron_news.12.csv,"Thu, 18 Oct 2001 15:10:12 -0700 (PDT)"
40,harris-s/deleted_items/103.,<15328247.1075858858567.JavaMail.evans@thyme>,"Thu, 18 Oct 2001 15:10:12 -0700 (PDT)",(no.address@enron.com),None,UPDATE - Supported Internet Email Addresses,1.0,text/plain; charset=us-ascii,7bit,Enron Global Technology@ENRON,...,,,"\SHARRIS1 (Non-Privileged)\Harris, Steven\Dele...",Harris-S,SHARRIS1 (Non-Privileged).pst,"\nEarlier this week, Enron Global Technology a...",harris-s,general_announcements,Ron.farmer-d.enron_news.12.csv,"Thu, 18 Oct 2001 15:10:12 -0700 (PDT)"
55,maggi-m/deleted_items/882.,<18317134.1075854991175.JavaMail.evans@thyme>,"Thu, 18 Oct 2001 15:10:12 -0700 (PDT)",(no.address@enron.com),None,UPDATE - Supported Internet Email Addresses,1.0,text/plain; charset=us-ascii,7bit,Enron Global Technology@ENRON,...,,,\MMAGGI (Non-Privileged)\Deleted Items,Maggi-M,MMAGGI (Non-Privileged).pst,"\nEarlier this week, Enron Global Technology a...",maggi-m,general_announcements,Ron.farmer-d.enron_news.12.csv,"Thu, 18 Oct 2001 15:10:12 -0700 (PDT)"
58,mccarty-d/deleted_items/16.,<32349775.1075852162297.JavaMail.evans@thyme>,"Thu, 18 Oct 2001 15:10:12 -0700 (PDT)",(no.address@enron.com),None,UPDATE - Supported Internet Email Addresses,1.0,text/plain; charset=us-ascii,7bit,Enron Global Technology@ENRON,...,,,"\DMCCARTY (Non-Privileged)\McCarty, Danny\Dele...",MCCARTY-D,DMCCARTY (Non-Privileged).pst,"\nEarlier this week, Enron Global Technology a...",mccarty-d,general_announcements,Ron.farmer-d.enron_news.12.csv,"Thu, 18 Oct 2001 15:10:12 -0700 (PDT)"
63,neal-s/deleted_items/551.,<934583.1075858826561.JavaMail.evans@thyme>,"Thu, 18 Oct 2001 15:10:12 -0700 (PDT)",(no.address@enron.com),None,UPDATE - Supported Internet Email Addresses,1.0,text/plain; charset=us-ascii,7bit,Enron Global Technology@ENRON,...,,,\SNEAL (Non-Privileged)\Deleted Items,Neal-S,SNEAL (Non-Privileged).pst,"\nEarlier this week, Enron Global Technology a...",neal-s,general_announcements,Ron.farmer-d.enron_news.12.csv,"Thu, 18 Oct 2001 15:10:12 -0700 (PDT)"
65,panus-s/deleted_items/32.,<7173643.1075858849324.JavaMail.evans@thyme>,"Thu, 18 Oct 2001 15:10:12 -0700 (PDT)",(no.address@enron.com),None,UPDATE - Supported Internet Email Addresses,1.0,text/plain; charset=us-ascii,7bit,Enron Global Technology@ENRON,...,,,"\SPANUS (Non-Privileged)\Panus, Stephanie\Dele...",Panus-S,SPANUS (Non-Privileged).pst,"\nEarlier this week, Enron Global Technology a...",panus-s,general_announcements,Ron.farmer-d.enron_news.12.csv,"Thu, 18 Oct 2001 15:10:12 -0700 (PDT)"
70,presto-k/deleted_items/590.,<26137745.1075852823354.JavaMail.evans@thyme>,"Thu, 18 Oct 2001 15:10:12 -0700 (PDT)",(no.address@enron.com),None,UPDATE - Supported Internet Email Addresses,1.0,text/plain; charset=us-ascii,7bit,Enron Global Technology@ENRON,...,,,\KPRESTO (Non-Privileged)\Deleted Items,Presto-K,KPRESTO (Non-Privileged).pst,"\nEarlier this week, Enron Global Technology a...",presto-k,general_announcements,Ro